# QV on Deneb

In [ ]:
%load_ext autoreload
%autoreload 2

## Choose Deneb as a backend

In [ ]:
backend = "deneb"

## Set IQM Token if using Resonance.

In [ ]:
import os
os.environ["IQM_TOKEN"] = ""

## Quantum Volume Configuration

In [ ]:
from iqm.benchmarks.quantum_volume.quantum_volume import *

In [ ]:
EXAMPLE_QV = QuantumVolumeConfiguration(
    num_circuits=500,
    shots=2**8,
    calset_id=None,
    num_sigmas=2,
    choose_qubits_routine="custom",
    custom_qubits_array=[[1,2,4]],  ### specify the layout of the qubits. 0 is reserved for the resonator.
    qiskit_optim_level=3,
    optimize_sqg=True,
    routing_method="sabre",
    physical_layout="fixed",
    max_gates_per_batch=60_000, ## this needs to be reduced when running for larger number of qubits
    rem=True,
    mit_shots=1_000,
)

## Run the experiment

In [ ]:
benchmark_qv = QuantumVolumeBenchmark(backend, EXAMPLE_QV)
run0_qv = benchmark_qv.run()

## Perform Analysis

In [ ]:
result0_qv = benchmark_qv.analyze()

List all the keys in the attributes of the dataset

In [ ]:
attr_keys = sorted([str(x) for x in list(result0_qv.dataset.attrs.keys())])
for k in attr_keys:
    print(k)

The data for specific experiments is indexed by 0,1,..., e.g., for the first experiment:

In [ ]:
for k in result0_qv.dataset.attrs[0].keys():
    print(k)

The observation object contains the benchmark results

In [ ]:
result0_qv.observations

Or print all the plots at once

In [ ]:
result0_qv.plot_all()

Store all the data of the run if one sees a successful quantum volume. 


In [ ]:
## Things needed to load the data. 
HOP = result0_qv.dataset.attrs[0]["REM_heavy_output_probabilities"]
HOP_cumul = result0_qv.dataset.attrs[0]["REM_cumulative_average_heavy_output_probability"]
HOP_cumul_std = result0_qv.dataset.attrs[0]["REM_cumulative_stddev_heavy_output_probability"]
HOP_threshold = [2.0/3.0]*len(HOP)

In [ ]:
# Create a directory to store the pickle files with a timestamp
import pickle
import os
from datetime import datetime

date_str = datetime.now().strftime("%Y-%m-%d")
save_dir = f"data_QV_{date_str}"
os.makedirs(save_dir, exist_ok=True)

# Function to save a list as a pickle file
def save_pickle(data, filename):
    with open(filename, "wb") as f:
        pickle.dump(data, f)

# Save each list separately
save_pickle(HOP, os.path.join(save_dir, "HOP.pkl"))
save_pickle(HOP_cumul, os.path.join(save_dir, "HOP_cumul.pkl"))
save_pickle(HOP_cumul_std, os.path.join(save_dir, "HOP_cumul_std.pkl"))
save_pickle(HOP_threshold, os.path.join(save_dir, "HOP_threshold.pkl"))

print(f"Pickle files saved in {save_dir}")